<a href="https://colab.research.google.com/github/iskra3138/ImageSr/blob/master/1_Image_Loading_Methods_%EB%B9%84%EA%B5%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Image를 loading 하는 몇 가지 방법에 대한 성능 비교를 위한 노트입니다.
- GPU를 사용하지 않으므로 가속기 없이 실행해도 됩니다.

In [1]:
## Tensorflow 2.X를 사용하면 첫번째 방법인 tf.io.read_file(path)의 속도가 더 빨라집니다. (나머지 세 방법은 큰 차이 없음)
%tensorflow_version 2.x

TensorFlow 2.x selected.


성능 방법 비교를 위한 Library들을 불러오고, 각 라이브러리의 버전을 확인합니다.

In [2]:
import tensorflow as tf
print ('tensorflow vesrion: {}'.format(tf.__version__))

tensorflow vesrion: 2.1.0


In [3]:
import cv2
print ('opencv vesrion: {}'.format(cv2.__version__))

opencv vesrion: 4.1.2


In [4]:
### pillow의 경우 일반적으로 numpy를 통해 array
import PIL
from PIL import Image
import numpy as np
print ('pillow version: {}'.format(PIL.__version__))
print ('numpy vesrion: {}'.format(np.__version__))

pillow version: 6.2.2
numpy vesrion: 1.17.5


test를 위해 아래 셀을 실행하고 이미지를 다운받습니다. (Flowers DataSet)

In [0]:
#@title Download Flowers Image Data [Run Me!!!]
import pathlib
data_dir = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
                                         fname='flower_photos', untar=True)
data_dir = pathlib.Path(data_dir)

path = '/root/.keras/datasets/flower_photos/daisy/100080576_f52e8ee070_n.jpg'

### 속도 측정
- 각 이미지를 로딩해서 numpy array로 만드는 데까지 걸리는 시간을 측정
- 다른 모든 방법의 output은 uint8이나, Keras API를 사용하면 결과값이 float32가 되므로 각 방법에 float변환 추가

In [0]:
import time

##### 1. tensorflow - tf.io.read_file(path)
- 첫 번째 실행방법은 tf.io.read_file을 사용하는 방법입니다.
- [tf.io.read_file](https://www.tensorflow.org/api_docs/python/tf/io/read_file)의 output은 string입니다.
- 본 예제는 jpeg파일을 사용하므로, array형태로 바꿔주기 위해서 [tf.image.decode_jpeg](https://www.tensorflow.org/api_docs/python/tf/io/decode_jpeg) 를 이용합니다. 


In [91]:
times = []
for i in range(1000):
  start = time.time()
  tf_io_img = tf.io.read_file(path) ## TFRecord 파일을 만들때는 여기까지만 있어도 되므로 더 빠름
  tf_io_img = tf.image.decode_jpeg(tf_io_img).numpy()
  tf_io_img = tf_io_img.astype(np.float32)
  end = time.time()
  times.append(end - start)
print ('average time: {:.3}'.format(sum(times) / len(times)))

average time: 0.00151


##### 2. tensorflow - keras
- keras api를 사용해서도 이미지를 불러올 수 있습니다
  - tf.keras.preprocessing.image.load_img 의 아웃풋은 piilow Image 객체가 되고
  - tf.keras.preprocessing.image.img_to_arra의 아웃풋은 float32가 됩니다.

In [93]:
times = []
for i in range(1000):
  start = time.time()
  keras_img = tf.keras.preprocessing.image.load_img(path)
  keras_img = tf.keras.preprocessing.image.img_to_array(keras_img)
  end = time.time()
  times.append(end - start)
print ('average time: {:.3}'.format(sum(times) / len(times)))

average time: 0.00233


##### 3. pillow

In [94]:
times = []
for i in range(1000):
  start = time.time()
  pil_img = Image.open(path)
  pil_npimg = np.array(pil_img)
  pil_npimg = pil_npimg.astype(np.float32)
  end = time.time()
  times.append(end - start)
print ('average time: {:.3}'.format(sum(times) / len(times)))

average time: 0.00218


##### 4. opencv
- opencv로 이미지를 불러오면 [BGR] 순서가 되므로, 반드시 [RGB] 순서로 변경해야 합니다.

In [95]:
times = []
for i in range(1000):
  start = time.time()
  cv_img = cv2.imread(path) # BGR
  cv_img = cv2.cvtColor(cv_img,cv2.COLOR_BGR2RGB)
  cv_img = cv_img.astype(np.float32)
  end = time.time()
  times.append(end - start)
print ('average time: {:.3}'.format(sum(times) / len(times)))

average time: 0.00193


tf.io.reasd_file을 쓸 때가 가장 빠르고, 그 다음이 opencv, pillow, tf.keras 순입니다.


| | tf.io.read_file | tf.keras |  pillow | opencv |
|----|----|----|----|----|
| avg.time | 0.00151 |0.00233|0.00218 | 0.00193 |
| 상대비교 | 1 | 1.54배 | 1.44 배 | 1.28배 |

### 결과값 비교

In [96]:
print (tf_io_img[0][0])
print (keras_img[0][0])
print (pil_npimg[0][0])
print (cv_img[0][0])

[133. 135. 132.]
[135. 135. 133.]
[135. 135. 133.]
[135. 135. 133.]


- **빠르기는 tf.io.read_file을 쓰는 것이 가장 빠르나, 혼자만 값이 다릅니다.**
  - 그러므로 tf.io.read_file을 사용해서 학습을 했으면, Prediction 에서도 tf.io.read_file를 사용해야 합니다!!!!